# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [581]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Ask for parameters

In [582]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [583]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Variables:

* **x:** Litres used of the first liquid component.
* **y:** Litres used of the second liquid component. 


# 2. Ask for objective

In [584]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [585]:
Markdown(response2.text)

Maximize:  ln(x*y) - (x + 2y)^(1/3) 


# 3. Ask for constraints

In [586]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [587]:
Markdown(response3.text)

## Constraints:

1. **Tank Capacity:** 
   *  x + y  ≤ 80 

2. **Stability Requirement:**
   * (x - (x+y)/2)^2 ≤ 10
   * (y - (x+y)/2)^2 ≤ 10 

This formulation captures the tank's maximum capacity and the stability requirement based on the squared difference from the mean. 


## 3. Generate the pyomo code

In [588]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [589]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
tank_capacity = 80
stability_limit = 10

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(domain=pyo.NonNegativeReals)
model.y = pyo.Var(domain=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr=pyo.log(model.x * model.y) - (model.x + 2*model.y)**(1/3), sense=pyo.maximize)

# Constraints
model.tank_constraint = pyo.Constraint(expr=model.x + model.y <= tank_capacity)
model.stability_x = pyo.Constraint(expr=(model.x - (model.x + model.y)/2)**2 <= stability_limit)
model.stability_y = pyo.Constraint(expr=(model.y - (model.x + model.y)/2)**2 <= stability_limit)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other nonlinear solvers as well
results = solver.solve(model)

# Display Results
print("Optimal Solution:")
print(f"x: {pyo.value(model.x)} Litres")
print(f"y: {pyo.value(model.y)} Litres")
print(f"Objective Value: {pyo.value(model.obj)}") 
```

**Explanation:**

1. **Import Pyomo:** We import the `pyomo.environ` module to access all Pyomo functionalities.
2. **Sample Data:**  I've included sample values for `tank_capacity` and `stability_limit`. You should replace these with the actual values from your problem. 
3. **Model Creation:** A concrete Pyomo model (`model`) is created.
4. **Variables:** Two non-negative continuous variables, `model.x` and `model.y`, are defined to represent the liters of each liquid component.
5. **Objective Function:** The objective function is defined to maximize `ln(x*y) - (x + 2y)^(1/3)`. 
6. **Constraints:**
   - `tank_constraint`:  Limits the total volume (x + y) based on the tank capacity.
   - `stability_x` and `stability_y`: Implement the stability requirements using squared differences from the mean volume.
7. **Solver:** We use the `ipopt` solver (a good choice for non-linear problems), but you might need to install it (`conda install -c conda-forge ipopt`)
8. **Results:** The optimal values of `x`, `y`, and the objective function are printed. 

Remember to replace the sample data with your actual values. 


## 4. Input problem data and try running the generated code

In [590]:
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
tank_capacity = 80
stability_limit = 10

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(domain=pyo.NonNegativeReals)
model.y = pyo.Var(domain=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr=pyo.log(model.x * model.y) - (model.x + 2*model.y)**(1/3), sense=pyo.maximize)

# Constraints
model.tank_constraint = pyo.Constraint(expr=model.x + model.y <= tank_capacity)
model.stability_x = pyo.Constraint(expr=(model.x - (model.x + model.y)/2)**2 <= stability_limit)
model.stability_y = pyo.Constraint(expr=(model.y - (model.x + model.y)/2)**2 <= stability_limit)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other nonlinear solvers as well
results = solver.solve(model)

# Display Results
print("Optimal Solution:")
print(f"x: {pyo.value(model.x)} Litres")
print(f"y: {pyo.value(model.y)} Litres")
print(f"Objective Value: {pyo.value(model.obj)}") 

Optimal Solution:
x: 43.162277233647174 Litres
y: 36.8377229631074 Litres
Objective Value: 2.4827783318454744


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [591]:
print(response.text)

### Variables:

* **x:** Litres used of the first liquid component.
* **y:** Litres used of the second liquid component. 



In [592]:
print(response2.text)

Maximize:  ln(x*y) - (x + 2y)^(1/3) 



In [593]:
print(response3.text)

## Constraints:

1. **Tank Capacity:** 
   *  x + y  ≤ 80 

2. **Stability Requirement:**
   * (x - (x+y)/2)^2 ≤ 10
   * (y - (x+y)/2)^2 ≤ 10 

This formulation captures the tank's maximum capacity and the stability requirement based on the squared difference from the mean. 



In [594]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
tank_capacity = 80
stability_limit = 10

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(domain=pyo.NonNegativeReals)
model.y = pyo.Var(domain=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr=pyo.log(model.x * model.y) - (model.x + 2*model.y)**(1/3), sense=pyo.maximize)

# Constraints
model.tank_constraint = pyo.Constraint(expr=model.x + model.y <= tank_capacity)
model.stability_x = pyo.Constraint(expr=(model.x - (model.x + model.y)/2)**2 <= stability_limit)
model.stability_y = pyo.Constraint(expr=(model.y - (model.x + model.y)/2)**2 <= stability_limit)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other nonlinear solvers as well
results = solver.solve(model)

# Display Results
print("Optimal Solution:")
print(f"x: {pyo.value(model.x)} Litres")
print(f"y: {pyo.value(model.y)} Litres")
print(f"Objective Value: {p